In [1]:
import os
import json
import pandas as pd

root_dir = (r'C:/Users/user/PycharmProjects/phonepe/data')

# Initialize empty list to hold dictionaries of data for each JSON file
data_list = []

# Loop over all the state folders
for state_dir in os.listdir(os.path.join(root_dir, 'aggregated/transaction/country/india/state')):
    state_path = os.path.join(root_dir, 'aggregated/transaction/country/india/state', state_dir)
    if os.path.isdir(state_path):
        
        # Loop over all the year folders
        for year_dir in os.listdir(state_path):
            year_path = os.path.join(state_path, year_dir)
            if os.path.isdir(year_path):
                
                # Loop over all the JSON files (one for each quarter)
                for json_file in os.listdir(year_path):
                    if json_file.endswith('.json'):
                        with open(os.path.join(year_path, json_file)) as f:
                            data = json.load(f)
                            
                            # Extract the data we're interested in
                            for transaction_data in data['data']['transactionData']:
                                row_dict = {
                                    'States': state_dir,
                                    'Transaction_Year': year_dir,
                                    'Quarters': int(json_file.split('.')[0]),
                                    'Transaction_Type': transaction_data['name'],
                                    'Transaction_Count': transaction_data['paymentInstruments'][0]['count'],
                                    'Transaction_Amount': transaction_data['paymentInstruments'][0]['amount']
                                }
                                data_list.append(row_dict)

# Convert list of dictionaries to dataframe
df = pd.DataFrame(data_list)

In [2]:
df.to_csv('aggregated_transaction.csv', index=False)
df = pd.read_csv('aggregated_transaction.csv')
df.head()

,States,Transaction_Year,Quarters,Transaction_Type,Transaction_Count,Transaction_Amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05


In [7]:
import pyodbc

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=ASHWADHAASINI\SQLEXPRESS;'
                      'Database=PHONEPE;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [4]:
cursor.execute('''
    CREATE TABLE Aggregatedtransactiondatatouse (
        States nvarchar(50),
        Transaction_Year int,
        Quarters int,
        Transaction_Type NVARCHAR(100),
        Transaction_Count int,
        Transaction_Amount money)
               ''')


ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'Aggregatedtransactiondatatouse' in the database. (2714) (SQLExecDirectW)")

In [ ]:
df = pd.read_sql('SELECT * FROM Aggregatedtransactiondatatouse', con=conn)

In [ ]:
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Aggregatedtransactiondatatouse (States, Transaction_Year, Quarters, Transaction_Type, Transaction_Count, Transaction_Amount)
                VALUES (?,?,?,?,?,?)
                ''',
                row.States, 
                row.Transaction_Year,
                row.Quarters,
                row.Transaction_Type,
                row.Transaction_Count,
                row.Transaction_Amount
                )
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM dbo.Aggregatedtransactiondatatouse")
rows = cursor.fetchall()
for row in rows:
    print(row)

In [11]:
df = pd.read_sql('SELECT * FROM dbo.Aggregatedtransactiondatatouse', con=conn)

In [78]:
query = "UPDATE dbo.Aggregatedtransactiondatatouse SET States = 'West Bengal' WHERE States = 'west-bengal'"
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [5]:
cursor.execute("SELECT * FROM dbo.Aggregatedtransactiondatatouse")
rows = cursor.fetchall()
for row in rows:
    print(row)

('Andaman & Nicobar Island', 2018, 1, 'Recharge & bill payments', 4200, Decimal('1845307.4674'))
('Andaman & Nicobar Island', 2018, 1, 'Peer-to-peer payments', 1871, Decimal('12138655.2997'))
('Andaman & Nicobar Island', 2018, 1, 'Merchant payments', 298, Decimal('452507.1686'))
('Andaman & Nicobar Island', 2018, 1, 'Financial Services', 33, Decimal('10601.4199'))
('Andaman & Nicobar Island', 2018, 1, 'Others', 256, Decimal('184689.8663'))
('Andaman & Nicobar Island', 2018, 2, 'Recharge & bill payments', 6735, Decimal('2320944.7311'))
('Andaman & Nicobar Island', 2018, 2, 'Peer-to-peer payments', 3575, Decimal('24511925.2310'))
('Andaman & Nicobar Island', 2018, 2, 'Merchant payments', 603, Decimal('1024491.0134'))
('Andaman & Nicobar Island', 2018, 2, 'Financial Services', 59, Decimal('121336.0271'))
('Andaman & Nicobar Island', 2018, 2, 'Others', 368, Decimal('359838.5054'))
('Andaman & Nicobar Island', 2018, 3, 'Recharge & bill payments', 8636, Decimal('4343504.8684'))
('Andaman & N

('Jharkhand', 2018, 4, 'Recharge & bill payments', 1885135, Decimal('328077093.4785'))
('Jharkhand', 2018, 4, 'Merchant payments', 845956, Decimal('373435992.4817'))
('Jharkhand', 2018, 4, 'Financial Services', 136918, Decimal('36357465.5224'))
('Jharkhand', 2018, 4, 'Others', 90561, Decimal('90026272.8389'))
('Jharkhand', 2019, 1, 'Peer-to-peer payments', 5952105, Decimal('12664709652.3471'))
('Jharkhand', 2019, 1, 'Recharge & bill payments', 2347079, Decimal('440323945.0577'))
('Jharkhand', 2019, 1, 'Merchant payments', 892826, Decimal('391616776.1122'))
('Jharkhand', 2019, 1, 'Financial Services', 105558, Decimal('47227410.8113'))
('Jharkhand', 2019, 1, 'Others', 90140, Decimal('83358664.9498'))
('Jharkhand', 2019, 2, 'Peer-to-peer payments', 5155937, Decimal('17044444613.8084'))
('Jharkhand', 2019, 2, 'Recharge & bill payments', 2734377, Decimal('562210324.8162'))
('Jharkhand', 2019, 2, 'Merchant payments', 2049311, Decimal('862186711.5217'))
('Jharkhand', 2019, 2, 'Financial Servi

('Nagaland', 2020, 4, 'Recharge & bill payments', 335297, Decimal('91929581.8773'))
('Nagaland', 2020, 4, 'Merchant payments', 90496, Decimal('113115660.1741'))
('Nagaland', 2020, 4, 'Financial Services', 1068, Decimal('1017278.3099'))
('Nagaland', 2020, 4, 'Others', 3571, Decimal('3194492.1527'))
('Nagaland', 2021, 1, 'Peer-to-peer payments', 477346, Decimal('2841021698.4845'))
('Nagaland', 2021, 1, 'Recharge & bill payments', 376473, Decimal('110095189.5932'))
('Nagaland', 2021, 1, 'Merchant payments', 116846, Decimal('151202427.1461'))
('Nagaland', 2021, 1, 'Financial Services', 1511, Decimal('1929643.6943'))
('Nagaland', 2021, 1, 'Others', 3721, Decimal('3201525.6952'))
('Nagaland', 2021, 2, 'Peer-to-peer payments', 526966, Decimal('2847731457.5715'))
('Nagaland', 2021, 2, 'Recharge & bill payments', 453979, Decimal('126962524.7454'))
('Nagaland', 2021, 2, 'Merchant payments', 144413, Decimal('176831583.1368'))
('Nagaland', 2021, 2, 'Financial Services', 1624, Decimal('1785915.6371

('Uttarakhand', 2020, 4, 'Merchant payments', 6629201, Decimal('4274032856.6465'))
('Uttarakhand', 2020, 4, 'Recharge & bill payments', 4140391, Decimal('1697819950.5565'))
('Uttarakhand', 2020, 4, 'Financial Services', 27120, Decimal('29791064.7618'))
('Uttarakhand', 2020, 4, 'Others', 68376, Decimal('42358455.0094'))
('Uttarakhand', 2021, 1, 'Merchant payments', 8393767, Decimal('5283142247.0658'))
('Uttarakhand', 2021, 1, 'Peer-to-peer payments', 7981710, Decimal('29202365442.6344'))
('Uttarakhand', 2021, 1, 'Recharge & bill payments', 4146569, Decimal('1727960484.0303'))
('Uttarakhand', 2021, 1, 'Financial Services', 30208, Decimal('38400863.9956'))
('Uttarakhand', 2021, 1, 'Others', 56148, Decimal('36320362.1723'))
('Uttarakhand', 2021, 2, 'Peer-to-peer payments', 9374939, Decimal('32674145244.3906'))
('Uttarakhand', 2021, 2, 'Merchant payments', 9134389, Decimal('6257528832.0572'))
('Uttarakhand', 2021, 2, 'Recharge & bill payments', 5388285, Decimal('1979824048.8157'))
('Uttarak

In [8]:

# Define the SQL query to fetch the column from the other table
query1 = "SELECT Registered_Users FROM dbo.Aggregateduserdatatouse"

# Fetch the data from the other table into a pandas dataframe
other_df = pd.read_sql(query1, connection)

# Define the SQL query to fetch data from the current table
query2 = "SELECT * FROM dbo.Aggregatedtransactiondatatouse"

# Fetch the data from the current table into a pandas dataframe
current_df = pd.read_sql(query2, connection)

# Join the other_df dataframe to the current_df dataframe using a common column
merged_df = pd.merge(current_df, other_df, on='States')

# Close the database connection
connection.close()

# Display the merged dataframe
print(merged_df.head())


NameError: name 'connection' is not defined